In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
from collections import defaultdict

In [3]:
kept_status = np.load("kept_status.npy")
lost_status = np.load("lost_status.npy")

In [4]:
kept_status, lost_status

(array(['DOV', 'TROW', 'T', 'NUE', 'GWW', 'DCI', 'CPKF', 'MKC', 'GD',
        'FUL', 'EFSI', 'ABM', 'WABC', 'WST', 'NIDB', 'RPM', 'PG', 'ATR',
        'LIN', 'MDU', 'CAT', 'UVV', 'SWK', 'TMP', 'SRCE', 'MCY', 'ITW',
        'LECO', 'ESS', 'UBSI', 'CVX', 'CFR', 'PPG', 'LOW', 'UMBF', 'NWN',
        'ARTNA', 'PSBQ', 'SYK', 'MDT', 'GRC', 'NC', 'FELE', 'BKH', 'FMCB',
        'NEE', 'EXPD', 'MO', 'WEYS', 'APD', 'KO', 'AOS', 'CINF', 'NFG',
        'CBU', 'SCL', 'ORI', 'OZK', 'UHT', 'SJW', 'FLIC', 'CBSH', 'EBTC',
        'BDX', 'WTRG', 'CTAS', 'ENB', 'GPC', 'CL', 'MCD', 'MGRC', 'IBM',
        'CHD', 'CSVI', 'TNC', 'FRT', 'PBCT', 'MATW', 'CLX', 'CTBI', 'JKHY',
        'BRO', 'MGEE', 'RNR', 'SPGI', 'TR', 'MSA', 'AFL', 'PII', 'O',
        'SEIC', 'TRI', 'BMI', 'WBA', 'PNR', 'ADP', 'SYY', 'KMB', 'CWT',
        'AROW', 'BEN', 'WMT', 'THFF', 'CNI', 'ATO', 'UGI', 'TDS', 'ADM',
        'NDSN', 'ERIE', 'JNJ', 'AWR', 'BRC', 'HRL', 'ALB', 'NNN', 'TGT',
        'XOM', 'SHW', 'BANF', 'MMM', 'MSEX', 'EMR', '

In [5]:
kept = defaultdict(None)
lost = defaultdict(None)

for tckr in kept_status:
    kept[tckr] = yf.Ticker(tckr)

for tckr in lost_status:
    lost[tckr] = yf.Ticker(tckr)

In [6]:
size = len(kept.keys()) + len(lost.keys())
size

172

Metrics
[T = [Ticker, Price, (Lost/Kept)]]

In [7]:
def re(t,status):
    years = []
    for i in range(len(t.dividends)):
        years.append(t.dividends.index[i].year)

    year, ct = np.unique(np.array(years), return_counts=True)
    # print(ct)

    fix_years = year[ct<4]
    # print(fix_years)
    
    df = t.dividends.to_frame().reset_index()

    if fix_years.size > 0:
        print("uhoh", t.ticker)
        for yr in fix_years:
            dont_fix_years = year[ct>4]
            if yr-1 in dont_fix_years or yr+1 in dont_fix_years:
                if ct[year==yr-1] > 4:
                    ct[year==yr-1] -= 1
                    ct[year==yr] += 1
                elif ct[year==yr+1] > 4:
                    ct[year==yr+1] -= 1
                    ct[year==yr] += 1
                continue
            years = []
            for i in range(len(df.Dividends)):
                years.append(df.Date[i].year)
            indx = df[years==yr].index[-1]
            for i in range(4-ct[year==yr][0]):
                val = 0 if status in "lost" else df.loc[indx].Dividends
                df.loc[indx + 0.5] = df.loc[indx].Date,val
                df = df.sort_index().reset_index(drop=True)
                indx += 1
    return df



In [8]:
for t in kept_status:
    try:
        tick = kept[t]
        tick.history(start='1900-1-1', end='2020-12-31')
        div = re(tick,"kept")
        div = pd.Series(div.Dividends.values, index=div.Date)
        if not div.empty:
            div.to_csv(f'data/series/good/{t}_dividends_fixed.csv')
    except Exception as e:
        print(f'Kept\n{e}')

for t in lost_status:
    try:
        tick = lost[t]
        tick.history(start='1900-1-1', end='2020-12-31')
        div = re(tick,"lost")
        div = pd.Series(div.Dividends.values, index=div.Date)
        if not div.empty:
            div.to_csv(f'data/series/good/{t}_dividends_fixed.csv')
    except Exception as e:
        print(f'Lost\n{e}')

uhoh DOV
uhoh TROW
uhoh T
uhoh NUE
uhoh CPKF
uhoh MKC
uhoh GD
uhoh FUL
uhoh EFSI
uhoh ABM
uhoh NIDB
uhoh RPM
uhoh ATR
uhoh LIN
uhoh MDU
uhoh CAT
uhoh UVV
uhoh SWK
uhoh TMP
uhoh SRCE
uhoh MCY
uhoh LECO
uhoh ESS
uhoh UBSI
uhoh CVX
uhoh CFR
uhoh PPG
uhoh LOW
uhoh UMBF
uhoh NWN
uhoh ARTNA
uhoh PSBQ
uhoh SYK
uhoh MDT
uhoh NC
uhoh FELE
uhoh FMCB
uhoh NEE
uhoh EXPD
uhoh MO
uhoh WEYS
uhoh APD
uhoh KO
uhoh CINF
uhoh NFG
uhoh ORI
uhoh SJW
uhoh FLIC
uhoh CBSH
uhoh EBTC
uhoh BDX
uhoh CTAS
uhoh ENB
uhoh GPC
uhoh CL
uhoh MCD
uhoh MGRC
uhoh IBM
uhoh CHD
uhoh CSVI
uhoh FRT
uhoh PBCT
uhoh MATW
uhoh CLX
uhoh CTBI
uhoh JKHY
uhoh RNR
uhoh SPGI
uhoh TR
uhoh MSA
uhoh AFL
uhoh PII
uhoh O


/var/folders/my/lq0q5bb97h3g3w8mx_7cxjf00000gn/T/ipykernel_81543/435813825.py:19: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if ct[year==yr-1] > 4:


uhoh SEIC
uhoh TRI
uhoh WBA
uhoh PNR
uhoh ADP
uhoh SYY
uhoh AROW
uhoh BEN
uhoh WMT
uhoh THFF
uhoh CNI
uhoh ATO
uhoh UGI
uhoh ADM
uhoh NDSN
uhoh ERIE
uhoh AWR
uhoh BRC
uhoh NNN
uhoh TGT
uhoh XOM
uhoh SHW
uhoh BANF
uhoh MMM
uhoh MSEX
uhoh EMR
uhoh ECL
uhoh RLI
uhoh ED
uhoh SON
uhoh LANC
uhoh CSL
uhoh PH
uhoh SBSI
uhoh NJR
uhoh ROP
uhoh PEP
uhoh VFC
- HGIC: 1d data not available for startTime=-2208970800 and endTime=1609390800. Only 100 years worth of day granularity data are allowed to be fetched per request.
- HGIC: No data found for this date range, symbol may be delisted
- HGIC: No data found for this date range, symbol may be delisted
Lost
'list' object has no attribute 'to_frame'
uhoh ABT
uhoh PBI
uhoh DBD
- WAG: 1d data not available for startTime=-2208970800 and endTime=1609390800. Only 100 years worth of day granularity data are allowed to be fetched per request.
- WAG: No data found for this date range, symbol may be delisted
- WAG: No data found for this date range, symbol may 